In [1]:
# Import library yang diperlukan
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

# Tentukan path ke database SQLite Anda
DATABASE_PATH = 'D:/workspace/EXPENSE_REPORT/flask2/expenses.db'

# Membuka koneksi ke database
conn = sqlite3.connect(DATABASE_PATH)

# Ambil data dari tabel Expense
query = "SELECT nominal, tanggal FROM expense"  # Pastikan nama tabelnya sesuai
df = pd.read_sql(query, conn)

# Pastikan kolom 'tanggal' sudah dalam format datetime
df['tanggal'] = pd.to_datetime(df['tanggal'])

# Menampilkan DataFrame
df.head()

# Tutup koneksi setelah selesai
conn.close()


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413 entries, 0 to 1412
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   nominal  1413 non-null   int64         
 1   tanggal  1413 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 22.2 KB


In [5]:
# Tambahkan kolom 'year_month' untuk grup bulanan
df['year_month'] = df['tanggal'].dt.to_period('M')

# Group by per bulan dengan agregasi sum
monthly_data = df.groupby('year_month')['nominal'].sum().reset_index()

# Konversi kembali 'year_month' menjadi datetime untuk perhitungan waktu
monthly_data['year_month'] = monthly_data['year_month'].dt.to_timestamp()

# Filter data 12 bulan terakhir
last_12_months = monthly_data[monthly_data['year_month'] >= (monthly_data['year_month'].max() - pd.DateOffset(months=12))]

# Hitung rata-rata dan median dari total nominal per bulan
average_12_months = last_12_months['nominal'].mean()
median_12_months = last_12_months['nominal'].median()

print(f"Average Monthly (12 months): {average_12_months:.2f}")
print(f"Median Monthly (12 months): {median_12_months:.2f}")

Average Monthly (12 months): 4377211.62
Median Monthly (12 months): 3173165.00


In [7]:
monthly_data['nominal'].sum()

np.int64(62576061)

In [8]:
last_12_months['nominal'].sum()

np.int64(56903751)

In [9]:
last_12_months

,year_month,nominal
3,2024-01-01,2334985
4,2024-02-01,3599582
5,2024-03-01,4269834
6,2024-04-01,4820467
7,2024-05-01,2989260
8,2024-06-01,2931760
9,2024-07-01,2739865
10,2024-08-01,18951749
11,2024-09-01,3329234
12,2024-10-01,3173165


In [10]:
monthly_data

,year_month,nominal
0,2023-10-01,1521275
1,2023-11-01,1457171
2,2023-12-01,2693864
3,2024-01-01,2334985
4,2024-02-01,3599582
5,2024-03-01,4269834
6,2024-04-01,4820467
7,2024-05-01,2989260
8,2024-06-01,2931760
9,2024-07-01,2739865


In [3]:
# Mendapatkan tanggal hari ini
today = datetime.today()

# Bulan ini
start_of_this_month = today.replace(day=1)
filtered_this_month = df[(df['tanggal'] >= start_of_this_month) & (df['tanggal'] < today)]

# Bulan kemarin
start_of_last_month = (start_of_this_month - pd.DateOffset(months=1)).replace(day=1)
end_of_last_month = start_of_this_month - pd.Timedelta(days=1)
filtered_last_month = df[(df['tanggal'] >= start_of_last_month) & (df['tanggal'] <= end_of_last_month)]



In [4]:
grouped_this_month = (
    filtered_this_month.groupby(filtered_this_month['tanggal'].dt.date)['nominal']
    .sum()
    .reset_index()
)

avg_daily = grouped_this_month['nominal'].mean()
avg_daily_rounded = round(avg_daily, 2)
print(f"Rata-rata pengeluaran per hari bulan ini: Rp{avg_daily_rounded}")

Rata-rata pengeluaran per hari bulan ini: Rp73960.81


In [5]:
# Hitung total nominal untuk bulan ini
total_this_month = filtered_this_month['nominal'].sum()

# Hitung total nominal untuk bulan sebelumnya
total_last_month = filtered_last_month['nominal'].sum()

# Hitung perubahan persentase dari bulan sebelumnya ke bulan ini
if total_last_month != 0:  # Pastikan tidak ada pembagian dengan nol
    change_in_month = ((total_this_month - total_last_month) / total_last_month) * 100
else:
    change_in_month = float('inf')  # Jika bulan sebelumnya 0, anggap perubahan tak terhingga

# Tampilkan hasil
# print(f"Total Nominal Bulan Ini: {total_this_month}")
# print(f"Total Nominal Bulan Sebelumnya: {total_last_month}")
print(f"Change in Month: {change_in_month:.2f}%")


Change in Month: -56.74%


In [8]:
change_in_month

np.float64(-56.7439811221388)

In [7]:
from datetime import timedelta

# Tanggal awal minggu ini dan minggu sebelumnya
start_of_this_week = today - timedelta(days=today.weekday())  # Senin minggu ini
end_of_this_week = start_of_this_week + timedelta(days=6)    # Minggu minggu ini
start_of_last_week = start_of_this_week - timedelta(days=7)  # Senin minggu sebelumnya
end_of_last_week = start_of_this_week - timedelta(days=1)    # Minggu minggu sebelumnya

# Filter data untuk minggu ini
filtered_this_week = df[(df['tanggal'] >= start_of_this_week) & (df['tanggal'] <= end_of_this_week)]

# Filter data untuk minggu sebelumnya
filtered_last_week = df[(df['tanggal'] >= start_of_last_week) & (df['tanggal'] <= end_of_last_week)]

# Hitung total nominal untuk minggu ini dan minggu sebelumnya
total_this_week = filtered_this_week['nominal'].sum()
total_last_week = filtered_last_week['nominal'].sum()

# Hitung perubahan persentase dari minggu sebelumnya ke minggu ini
if total_last_week != 0:  # Hindari pembagian dengan nol
    change_in_week = ((total_this_week - total_last_week) / total_last_week) * 100
else:
    change_in_week = float('inf')  # Jika minggu sebelumnya 0, perubahan dianggap tak terhingga

# Tampilkan hasil
print(f"Total Nominal Minggu Ini: {total_this_week}")
print(f"Total Nominal Minggu Sebelumnya: {total_last_week}")
print(f"Change in Week: {change_in_week:.2f}%")


Total Nominal Minggu Ini: 0
Total Nominal Minggu Sebelumnya: 199000
Change in Week: -100.00%


In [6]:
today_expenses = df[df['tanggal'] >= today]['nominal'].sum()
print("Today's Expenses: ", today_expenses)

Today's Expenses:  0


In [ ]:
# Data untuk dikembalikan ke frontend
expenses = {
    "month": total_this_month,
    "today": today_expenses,
    "change_in_week": change_in_week,
    "change_in_month": change_in_month,
    "avg_daily_month": avg_daily_rounded
}

In [9]:
filtered_last_week

,nominal,tanggal
1399,9200,2025-01-21
1400,25600,2025-01-21
1401,17000,2025-01-21
1402,22000,2025-01-21
1403,12000,2025-01-22
1404,15000,2025-01-22
1405,13000,2025-01-23
1406,19000,2025-01-23
1407,17000,2025-01-23
1408,5000,2025-01-23


In [10]:
print("start last week",start_of_last_week)
print("end last week",end_of_last_week)

start last week 2025-01-20 22:03:44.777145
end last week 2025-01-26 22:03:44.777145


In [11]:
total_this_week

np.int64(0)